In [ ]:
#Importing required libaries for code to run

%matplotlib notebook

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from cartopy.feature import ShapelyFeature
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import fiona
import os
from shapely.geometry import Point, LineString, Polygon


plt.ion() # makes the plotting of the map interactive

# generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

# create a scale bar of length 20 km in the upper right corner of the map
def scale_bar(ax, location=(0.92, 0.95)):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]

    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    plt.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=tmc)
    plt.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=tmc)
    plt.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=tmc)

    plt.text(sbx, sby-4500, '20 km', transform=tmc, fontsize=8)
    plt.text(sbx-12500, sby-4500, '10 km', transform=tmc, fontsize=8)
    plt.text(sbx-24500, sby-4500, '0 km', transform=tmc, fontsize=8)
    
#Create variable for open spaces dataset and load data

openspace_data = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\NS_GreenspaceSite.shp')

#Create variable for boundary dataset and load data, this example uses Glasgow as an area of interest

boundary_data = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\GlasgowBoundary.shp')

#Create variable for roads dataset and load data

road_data = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\NS_RoadLink.shp')

#Create Variable for glasgow postcode data 

postcode_g = gpd.read_file (r'C:\Users\angel\Programming\Project\Data\g_postcode_data.shp')


In [ ]:
#display open space dataset table

openspace_data

In [ ]:
#Display Boundary dataset table

boundary_data

In [ ]:
#Display road dataset table 

road_data

In [ ]:
#display postcode dataset table

postcode_g

In [ ]:
# Check co-ordinate reference systems for openspace layers

openspace_data.crs

In [ ]:
# Check co-ordinate reference systems for boundary layer

boundary_data.crs

In [ ]:
# Check co-ordinate reference systems for roads layer

road_data.crs

In [ ]:
#check co-ordinate reference systsem for postcode layer 

postcode_g.crs

In [ ]:
#Clip the open space dataset to the extent of the project boundary layer, using geopandas clip function 

# Clip data
openspace_glasgow = gpd.clip(openspace_data, boundary_data)

# Ignore missing/empty geometries
openspace_glasgow = openspace_glasgow[~openspace_glasgow.is_empty]

#Print number of rows in new clipped dataset compared to original dataset. Ensures that clip has worked.
print("The clipped data has fewer polygons (represented by rows):",
      openspace_glasgow.shape, openspace_data.shape)

#export clipped open space dataset into ESRI shapefile into data folder

openspace_glasgow.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\openspace_glasgow.shp', driver = 'ESRI Shapefile')


In [ ]:
#Clip the roads dataset to the extent of the project boundary layer, using geopandas clip function 

# Clip data
road_glasgow = gpd.clip(road_data, boundary_data)

# Ignore missing/empty geometries
road_glasgow = road_glasgow[~road_glasgow.is_empty]

#Print number of rows in new clipped dataset compared to original dataset. Ensures that clip has worked.
print("The clipped data has fewer line sections (represented by rows):",
      road_glasgow.shape, road_data.shape)

#export clipped road dataset into ESRI Shapefile into data folder

road_glasgow.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\road_glasgow.shp', driver = 'ESRI Shapefile')


In [ ]:
# identify the number of open space functions in the dataset to identify how many colours are required for symboloisation 
num_openspace = len(openspace_glasgow.function.unique())
print('Number of unique Open Space Types: {}'.format(num_openspace))
print(openspace_glasgow['function'].unique())

In [ ]:
# Identify the number of unique road types in the roads datasetto identify how many colours are required for symboloisation 
num_road = len(road_glasgow.function.unique())
print('Number of unique road classes: {}'.format(num_road)) 

In [ ]:
myFig = plt.figure(figsize=(10,10))  # create a figure of size 10x10 (representing the page size in inches)

myCRS = ccrs.UTM(30)  # create a Universal Transverse Mercator reference system to transform our data.

ax = plt.axes(projection=ccrs.Mercator())  # finally, create an axes object in the figure, using a Mercator projection


In [ ]:
# adding the outline of Glasgow City Boundary using cartopy's ShapelyFeature
outline_feature = ShapelyFeature(boundary_data['geometry'], myCRS, edgecolor='yellow', facecolor='w')
xmin, ymin, xmax, ymax = boundary_data.total_bounds
ax.add_feature(outline_feature) # add the boundary feature to the map

# using the boundary of the shapefile features, zoom the map to our area of interest
ax.set_extent([xmin, xmax, ymin, ymax], crs=myCRS) 

#add gridlines to the map, turning off the top and rightside labels 
gridlines = ax.gridlines(draw_labels=True, color='black', alpha=0.6, linestyle='--')
gridlines.right_labels =False # turn off the right side labels
gridlines.top_labels =False # turn off the top labels

In [ ]:
#add title to map figure
plt.title('Glasgow Open Spaces')

In [ ]:
#create colours for the open space types - for this dataset 10 colours need to be identified

openspace_colors = ['lightgreen','palevioletred', 'crimson', 'dimgrey', 'lime', 'darkorchid', 'darkorange', 'hotpink', 'indigo', 'aqua'  ]

# get a list of unique names for the Open Space type
openspace_types = list(openspace_glasgow.function.unique())

# sort the open space types alphabetically by name
openspace_types.sort() 

# add the open spaces data to the map

for i, openspace in enumerate(openspace_types):
    feat = ShapelyFeature(openspace_glasgow['geometry'][openspace_glasgow['function'] == openspace], myCRS, 
                        edgecolor='black',
                        facecolor=openspace_colors[i],
                        linewidth=1,
                        alpha=0.25)
    ax.add_feature(feat)


In [ ]:
#add roads layer to map 
road_colors = ['darkslategrey', 'navy', 'silver', 'darkmagenta', 'sienna', 'darkred', 'darkgoldenrod', 'olive']

# get a list of unique road types from the function attribute for the roads dataset
road_types = list(road_glasgow.function.unique())

# sort the open space types alphabetically by name
road_types.sort() 

# add the road data to the map
for i, road in enumerate(road_types):
    road_feat = ShapelyFeature(road_glasgow['geometry'][road_glasgow['function'] == road], myCRS, 
                                edgecolor='black',
                                facecolor=road_colors[i],
                                linewidth=0.25)
    ax.add_feature(road_feat)
    
myFig # to show the updated figure

In [ ]:
# generate a list of handles for the openspace dataset
#openspace_handles = generate_handles(openspace_glasgow.function.unique(), openspace_colors, alpha=0.25)

#  generate a list of handles for the road dataset
#road_handles = generate_handles(road_glasgow.function.unique(), road_colors, alpha=0.25)

# note: if you change the color you use to display rivers, you'll want to change it here, too
#boundary_handle = [mpoly.poly2D([], [], edgecolor='yellow', facecolor='w)]

# update county_names to take it out of uppercase text
#openspace_types = [function.title() for function in openspace_glasgow]
                                
# update county_names to take it out of uppercase text
#road_names = [function.title() for function in road_glasgow]

#ax.legend() takes a list of handles and a list of labels corresponding to the objects you want to add to the legend
#handles = openspace_handles #+ road_handles
                                
#labels = openspace_types #+ road_names 

#leg = ax.legend(handles, labels, title='Legend', title_fontsize=12, 
               #  fontsize=10, loc='upper right', frameon=True, framealpha=1)

#myFig # to show the updated figure

In [ ]:
# Create a new column within the openspace glasgow called area and populate it with the area in m2 for each 
# Area / 1000 to get total area in km2
openspace_glasgow['area km'] = openspace_glasgow.area/1000

In [ ]:
#Display the openspace glasgow table with the added areas column
openspace_glasgow

In [ ]:
#run the a groupby with count operation on the glasgow road layer to identify the number of each type of road in the glasgow area
openspace_groupcount = openspace_glasgow.groupby('function')['function'].count()

#Display the open space group with count by table
openspace_groupcount

In [ ]:
#run the a group by operation on the glasgow open space layer to identify the number of each type of open space in the glasgow area
openspace_group = openspace_glasgow.groupby('function')

#Display the open space group by table
openspace_group

#Print to show user that group 
print('Groupby Successful')

In [ ]:
#for key values in the open space groups create indivadual tables
for key, values in openspace_group:
    openspace_type = values

openspace_type
#display a sample table of the openspace table seperated by key values 

In [ ]:
#Create individual shapefiles for the types of openspace data. 
#determind output folder location
outFolder = r'C:\Users\angel\Programming\Project\Data'

# Create a new folder called 'results' (if does not exist already) to cretae the folder use os.makedirs() function
resultFolder = os.path.join(outFolder, 'results')
if not os.path.exists(resultFolder):
    os.makedirs(resultFolder)

# Iterate over the key values in the open space group to create seperate shapefiles for each
for key, values in openspace_group:
    # Format the filename (replace spaces with underscores)
    outName = "%s.shp" % key.replace(" ", "_")

    # Print some information for the user
    print("Processing: %s" % key)

    # Create an output path
    outpath = os.path.join(resultFolder, outName)

    # Export the data
    values.to_file(outpath)
    
    #print finishing statement to ensure shapefiles have be saved
    print('Shapefiles Saved') 
    

In [ ]:
#display the length of each road section in the road dataset
road_glasgow.length

In [ ]:
# Create a new column within the road_glasgow called length and populate it with the length in meters for each 
road_glasgow['length - m'] = road_glasgow.length

In [ ]:
#display the updated road glasgow with the length field included in the table 
road_glasgow

In [ ]:
#run the a group by operation on the glasgow open space layer to identify the number of each type of open space in the glasgow area
road_groupcount = road_glasgow.groupby('function')['function'].count()

#Display the open space group with count by table
road_groupcount

In [ ]:
#run the a group by operation on the glasgow road layer to identify the number of each type of open space in the glasgow area
road_group = road_glasgow.groupby('function')

#Display the open space group by table
road_group
print('Groupby Successful')

In [ ]:
#Create individual shapefiles for the types of road data. 
#determind output folder location
outFolder = r'C:\Users\angel\Programming\Project\Data'

# Create a new folder called 'Results' (if does not exist) to that folder using os.makedirs() function
resultFolder = os.path.join(outFolder, 'results')
if not os.path.exists(resultFolder):
    os.makedirs(resultFolder)

# Iterate over the
for key, values in road_group:
    # Format the filename (replace spaces with underscores)
    outName = "%s.shp" % key.replace(" ", "_")

    # Print some information for the user
    print("Processing: %s" % key)

    # Create an output path
    outpath = os.path.join(resultFolder, outName)

    # Export the data
    values.to_file(outpath)
    
    #print finishing statement to ensure shapefiles have be saved
    print('Shapefiles Saved')

In [ ]:
#Call in the new shapefiles which are going to be use in the interegation intergation

#public spaces
openspace_public = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\results\Public_Park_Or_Garden.shp')

#playing field
openspace_field = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\results\Playing_Field.shp')

#play space
openspace_play = gpd.read_file(r'C:\Users\angel\Programming\Project\Data\results\Play_Space.shp')


In [ ]:
#buffer open space polygons

#create 100m buffer around the public open spaces
openpublic_100 = openspace_public.buffer(100)

#create 100m buffer around playing fields
openfield_100 = openspace_field.buffer(100)

#create 100m buffer aroung play spaces 
openplay_100 = openspace_play.buffer(100)

#save 100m buffer of open space public to file
openpublic_100.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\openpublic_100m_buffer.shp', driver = 'ESRI Shapefile')

#save 100m buffer of open space public to file
openfield_100.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\openfield_100m_buffer.shp', driver = 'ESRI Shapefile')

#save 100m buffer of open space public to file
openplay_100.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\openplay_100m_buffer.shp', driver = 'ESRI Shapefile')

#print statement to ensure save has worked 
print ('Save Complete')

In [ ]:
# clip g_postcode_data to glasgow city council boundary
# Clip data
postcode_glasgow = gpd.clip(postcode_g, boundary_data)

# Ignore missing/empty geometries
postcode_glasgow = postcode_glasgow[~postcode_glasgow.is_empty]

#save glasgow postcode data to file
postcode_glasgow.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\postcode_glasgow.shp', driver = 'ESRI Shapefile')

In [ ]:
#clip postcode data to those inside the 100m buffer of the open space

# Clip data
postcode_openspace100 = gpd.clip(postcode_glasgow, openspace_100)

# Ignore missing/empty geometries
postcode_openspace100 = postcode_openspace100[~postcode_openspace100.is_empty]

#save glasgow postcode data to file
postcode_glasgow100.to_file('C:\\Users\\angel\\Programming\\Project\\Data\\postcode_glasgow.shp', driver = 'ESRI Shapefile')